# Setting up an OpenMM iMD simulation

This notebook demonstrates how to set up an OpenMM simulation for use with Narupa from scratch.
We take AMBER files for neuraminidase with oseltamivir (AKA tamiflu) bound, create an OpenMM system and 
set it up with Narupa, using ASE as the integrator

## Set up the OpenMM Simulation

We start by creating an OpenMM simulation from our AMBER files. OpenMM also supports Gromacs and CHARMM files, and can be customized for many other uses. 

In [ ]:
import simtk.openmm as mm
import simtk.unit as unit 
import simtk.openmm.app as app

In [ ]:
prmtop = app.AmberPrmtopFile("openmm_files/3TI6_ose_wt.top")
amber_coords = app.AmberInpcrdFile("openmm_files/3TI6_ose_wt.rst")

Because we use ASE for integrating, we keep the simulation simple by using implicit solvent and no constraints

In [ ]:
system = prmtop.createSystem(nonbondedMethod=app.CutoffPeriodic, 
                             nonbondedCutoff=2*unit.nanometer, 
                             implicitSolvent=app.OBC2,
                             constraints=None)

In [ ]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.001*unit.picoseconds)

Create an OpenMM simulation out of the topology, system and integrator

In [ ]:
simulation = app.Simulation(prmtop.topology, system, integrator)

In [ ]:
simulation.context.setPositions(amber_coords.positions)
if amber_coords.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*amber_coords.boxVectors)

Minimize the energy to create a stable conformation

In [ ]:
simulation.minimizeEnergy()

Run a few steps to check it's stable

In [ ]:
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)

In [ ]:
import sys

In [ ]:
simulation.reporters.append(app.StateDataReporter(sys.stdout, 100, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(1000)

Looks good! Now, let's set it up for use with Narupa.

The following cell outputs the system as an OpenMM XML file + a PDB file with the topology. This let's you take what you've done here and use it straight in Narupa elsewhere, perfect if you just want to run a simulation quickly: 

```bash 
narupa-omm-ase neuraminidase_narupa.xml
```

In [ ]:
from narupa.openmm.serializer import serialize_simulation

with open('neuraminidase_narupa.xml','w') as f:
    f.write(serialize_simulation(simulation))

In the following section, we'll set up the simulation by hand, essentially replicating what `narupa-omm-ase` does.

## Setting up an OpenMM simulation with ASE and Narupa

Now that we've got an OpenMM simulation, let's pair it with ASE so we can do interactive molecular dynamics with Narupa.

**Note**: Soon, this will be a lot simpler. 

At the time of writing, we use a custom [calculator](https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html#module-ase.calculators) to allow us to use OpenMM as the forcefield, and ASE as the integrator. 
This is so we can do the interactive forces easily in python. 
However, in an [upcoming release](https://gitlab.com/intangiblerealities/narupa-protocol/-/tree/feature/python-vmd-imd), we will implement the [interactive forces directly in OpenMM](https://gitlab.com/intangiblerealities/narupaplugins/openmm-vmd-imd), so all of the speed and features of OpenMM (such as constraints) will become available. Stay tuned!

### Creating the ASE System

We use the OpenMMCalculator to take our simulation and produce an ASE Atoms object

In [ ]:
from narupa.ase.openmm import OpenMMCalculator

In [ ]:
calculator = OpenMMCalculator(simulation)

In [ ]:
atoms = calculator.generate_atoms()
atoms.set_calculator(calculator)
len(atoms)

Now we've got an ASE Atoms object, and a calculator, we can set up Narupa with ASE as [usual](./basic_example.ipynb). 
The only difference here is that we swap out the default way of sending frames with a specially made one, `openmm_ase_frame_adaptor`, for OpenMM that knows about OpenMM topology

In [ ]:
from ase.md import Langevin
import ase.units as ase_units
dynamics = Langevin(atoms, timestep=1.0 * ase_units.fs, temperature=300 * ase_units.kB, friction=1.0e-03)

In [ ]:
from narupa.ase import NarupaASEDynamics
from narupa.app import NarupaImdApplication
from narupa.ase.openmm.runner import openmm_ase_frame_adaptor

In [ ]:
narupa_server = NarupaImdApplication.basic_server(port=0)
imd = NarupaASEDynamics(narupa_server,dynamics, frame_method=openmm_ase_frame_adaptor)

In [ ]:
imd.atoms.get_calculator()

As always, let's run a few steps to make sure everything looks good

In [ ]:
imd.run(10)

In [ ]:
imd.atoms.get_potential_energy()

All good! Let's leave it running in the background

In [ ]:
imd.run()

Connect to it from VR and you'll see something like this:

![Narupa neuraminidase](./images/neuraminidase_ball_and_stick.png)

## Let's make it pretty!

Ball and stick is so 2001, let's make it look cool. We'll also make it so if you interact with the oseltamivir, it'll be interacted with as a group, which is more stable

First, we connect a client so we can modify the shared state

In [ ]:
from narupa.app import NarupaImdClient
client = NarupaImdClient.connect_to_single_server(port=narupa_server.port)
client.subscribe_to_frames()
client.wait_until_first_frame();

We define a couple of handy methods for playing with selections and colour gradients

In [ ]:
import matplotlib.cm

def get_matplotlib_gradient(name: str):
    cmap = matplotlib.cm.get_cmap(name)
    return list(list(cmap(x/7)) for x in range(0, 8, 1))

In [ ]:
from narupa.mdanalysis import frame_data_to_mdanalysis
def generate_mdanalysis_selection(selection: str):
    universe = frame_data_to_mdanalysis(client.first_frame)
    idx_array = universe.select_atoms(selection).indices
    return map(int, idx_array)

Hide the 'root' selection, it's getting in the way of our creativity 

In [ ]:
root_selection = client.root_selection
with root_selection.modify():
    root_selection.hide = True
    root_selection.interaction_method = 'none'

Let's select the protein

In [ ]:
protein = client.create_selection("Protein", [])

In [ ]:
with protein.modify():
    protein.set_particles(generate_mdanalysis_selection("protein and not type H"))

We'll colour it and render with a spline, or ribbon, renderer.
Some things you can try: 
* Change the render: `spline`, `geometric spline`. Or comment out the `sequence` line and try `liquorice`,`noodles`, `cycles`, `ball and stick`.
* Change the color: set it to be one color, or try some different matplotlib [color maps](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html), e.g. `rainbow` or `magma`.
* Change the scale.

In [ ]:
with protein.modify():
    protein.renderer = {
            'sequence': 'polypeptide',
            'color': {
                'type': 'residue index in entity',
                'gradient': get_matplotlib_gradient('rainbow')
            },
            'render': 'geometric spline',
            'scale': 0.2
        }
    protein.interaction_method = 'single'

Let's reintroduce the ligand, oseltamivir, and make it so we interact with it as a group 

In [ ]:
# Select ligand
ligand = client.create_selection("Ligand", [])
with ligand.modify():
    ligand.set_particles(generate_mdanalysis_selection("resname OSE"))

In [ ]:
with ligand.modify():
    ligand.renderer = {
            'color': 'cpk',
            'scale': 0.1,
            'render': 'liquorice'
        }
    ligand.velocity_reset = True
    ligand.interaction_method = 'group'

If you've done all that, you'll have something that looks like this:

![Neuraminidase Geometric](./images/neuraminidase_geometric_spline.png)

# Tidying Up After Yourself

In [ ]:
client.close()

In [ ]:
imd.close()
narupa_server.close()

# Next Steps

* Set up an OpenMM simulation of [graphene](./openmm_graphene.ipynb) with restraints and add UI and custom commands in the notebook 